# **Курс "Програмування на мові Python"**

## **Практичне зайняття №16**

### Тема: "Декоратори"

Декоратори використовуються у ситуаціях, коли до функції потрібно "додати додаткову поведінку" ("add additional behaviour"). У мові Python декоратори - це функції, що приймають на вхід інші функції (або інші об'єкти) та повертають нові функції, які мають вже "декоровану" поведінку.

**Декоратор** - це функція, яка дозволяє загорнути (to wrap) іншу функцію, щоб розширити її функціональність, не змінюючи вихідного коду, та повертає нову функцію.

Щоб визначити декоратор, необхідно визначити функцію, яка прийматиме на вхід іншу функцію, як параметр, та повертатиме нову функцію. Наприклад:

In [ ]:
def logger(func):
    def inner():
        print('calling ', func.__name__)
        func()
        print('called ', func.__name__)
    return inner

У цьому випадку декоратор logger() загортає функцію func() у функцію inner(). Функція inner() буде розширювати можливості функції func(), роздруковуючи додатково два рядки. Декоратор повертає функцію inner(), як результат.

Кожна функція має атрибут `__name__`, що повертає назву функції. Він використовується всередині функції inner(), щоб вивести назву функції, яка фактично буде виконуватись.

Розглянемо результат роботи декоратора. Для цього визначимо функцію target():

In [ ]:
def target():
    print('In target function')

Тепер застосуємо декоратор logger() до функції target() та присвоїмо результат роботи декоратора змінній з такою ж назвою - target.

In [ ]:
target = logger(target)
target()

Під час виконання цього коду фактично виконується функція inner(), яку повертає декоратор. Функція inner() роздруковує перше повідомлення та викликає функцію, яка прийшла на вхід декоратора logger(). Після виконання цієї функції роздруковується друге повідомлення. Декоратор logger() повертає функцію inner(). Під час виклику декоратора відбувається підміна функції target(). Замість оригінальної її версії тепер фактично викликатиметься функція inner().

Однак у мові Python є можливість спростити синтаксис для виклику декоратора. Щоб навісити декоратор на функцію, достатньо написати його назву після символу '@' перед заголовком функції.

In [ ]:
@logger
def target():
    print('In target function')

target()

Декоратори також можуть застосовуватись до функцій, які приймають на вхід певні параметри. Але в такому випадку функція inner() має приймати на вхід стільки ж параметрів, скільки й функції, до яких декоратор застосовується. Наприклад:

In [ ]:
def logger(func):
    def inner(x, y):
        print('calling', func.__name__, 'with', x, 'and', y)
        func(x, y)
        print('returned from', func.__name__)
    return inner

@logger
def my_func(x, y):
    print(x, y)

my_func(4, 5)

Але для того, щоб зробити декоратор більш універсальним, тобто щоб його можна було застосовувати до функцій з різною кількістю параметрів, його можна оформити так:

In [ ]:
def logger(func):
    def inner(*args, **kwargs):
        print('*args parameters:', args)
        print('**kwargs parameters:', kwargs)
        res = func(*args, **kwargs)
        print(res)
        print()
    return inner

Тепер створимо дві функції з різною кількістю параметрів та застосуємо до них декоратор logger().

In [ ]:
@logger
def func_1(x, y, z):
    return x + y + z

@logger
def func_2(x, y):
    return x * y

func_1(2, 3, z = 1)
func_2(3, 4)

Бачимо, що позиційні аргументи передались у змінну args, а іменовані - у змінну kwargs. Бажано використовувати саме такий універсальний запис для декоратора навіть в тому випадку, якщо фукції, які подаються на вхід, аргументів не мають.

Декоратори можуть бути **складеними** (stacked decorators). Тобто до функції може бути застосовано більше одного декоратора. Наприклад:

In [ ]:
# Define the decorator functions
def make_bold(fn):
    def makebold_wrapped():
        return "<b>" + fn() + "</b>"
    return makebold_wrapped

def make_italic(fn):
    def makeitalic_wrapped():
        return "<i>" + fn() + "</i>"
    return makeitalic_wrapped

# Apply decorators to function hello
@make_bold
@make_italic
def hello():
    return 'hello world'

# Call function hello
print(hello())

Декоратори застосовуються до функції у послідовності, починаючи з останнього, тобто починаючи від того, що розташований найближче до заголовку функції. У попередньому прикладі спочатку виконується декоратор make_italic(), потім make_bold().

Декоратори також можуть приймати на вхід параметри та повертати функцію, що використовуватиме ці параметри. Наприклад:

In [ ]:
def register(active=True):
    def wrap(func):
        def wrapper(*args, **kwargs):
            print('Calling ', func.__name__, ' decorator param', active)
            if active:
                func(*args, **kwargs)
                print('Called ', func.__name__)
            else:
                print('Skipped ', func.__name__)
        return wrapper
    return wrap

@register()
def func1(x, y):
    print('x + y =', x + y)
    print('func1')

@register(active=False)
def func2():
    print('func2')

func1(1, 2)
print('-' * 10)
func2()

Першою відмінністю декораторів з параметрами від звичайних декораторів є додаткова функція (у попередньому прикладі - register()), яка огортає функцію wrap(), що приймає на вхід функцію-аргумент, та функцію wrapper(), яка приймає на вхід аргументи функції-аргумента.

Другою відмінністю є необхідність використовувати круглі дужки під час виклику декоратора навіть в тому випадку, якщо аргументи не передаються.

На відміну від звичайних функцій, функції-декоратори запускаються відразу після визначення. Наприклад:

In [ ]:
def logger(func):
    print('In Logger')
    def inner():
        print('In inner calling ', func.__name__)
        func()
        print('In inner called ', func.__name__)
    print('Finishing Logger')
    return inner

@logger
def print_it():
    print('Print It')

print('Start')
print_it()
print('Done')

З попереднього прикладу видно, що рядки "In Logger" та "Finishing Logger" виводяться ще до того, як функція print_it() була запущена.

У мові Python існує дуже багато **вбудованих декораторів**. Розглянемо найпростіший.

За замовчуванням під час виклику декоратора назва та рядок документації оригінальної функції втрачаються. Якщо функція inner() має рядок документації, то під час виклику декорованої функції буде виведений саме цей рядок. Наприклад:

In [ ]:
def logger(func):
    def inner():
        """inner() function docstring"""
        print('calling ', func.__name__)
        func()
        print('called ', func.__name__)
    return inner

@logger
def get_text(name):
    """get_text() function docstring"""
    return "Hello "+ name

print('name:', get_text.__name__)
print('doc: ', get_text.__doc__)

Щоб отримати рядок документації декорованої функції, можна скористатись декоратором wraps з модуля functools. Його потрібно ввести перед заголовком функції inner() та передати йому як параметр функцію func.

In [ ]:
from functools import wraps

def logger(func):
    @wraps(func)
    def inner():
        """inner() function docstring"""
        print('calling ', func.__name__)
        func()
        print('called ', func.__name__)
    return inner

@logger
def get_text(name):
    """get_text() function docstring"""
    return "Hello "+name

print('name:', get_text.__name__)
print('doc: ', get_text.__doc__)